## 최종 백엔드 코드_API 테스트 완료
'최종 백엔드 코드_람다함수 테스트 완료' 코드가 제대로 반환이 안되면 해당 코드를 사용해주세요.



## 수정된 부분

1) 람다 핸들러 함수: 쿼리스트링으로 가져올 때, get에서 'queryStringParameters'사용

2) 람다 핸들러 함수: return값을 header, body 형식에 맞춰 수정

3) 예시 호출 코드: 위의 수정사항과 맞게 변형

*람다함수 내 테스트 코드에서는 {"queryStringParameters": {~}} 이 부분만 넣어주시면 됩니다*

## 알림

1) install 무시

2) dynamoDB 인증설정은 모든 코드 맨 처음에 복붙해주세요

3) API 문서 순서대로 코드가 기입되어있습니다

4) 코드 맨 밑에 포함된 *# 테스트 코드* 는 주석처리하여 올려주세요.

테스트코드는 수정된 데이터로 바꾼 후 테스터로 사용하셔도 됩니디.

In [ ]:
# boto3 설치
!pip install boto3


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 66.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.6/82.6 kB 6.4 MB/s eta 0:00:00


## 모든 코드 제일 위에 아래 코드를 복붙해주세요

dynamoDB 인증 설정 코드

In [ ]:
# 인증코드를 복붙해주세요요

## 1. GET_SEARCH

In [ ]:
### lambda_handler에서 return 형식 맞춰야함 ###


### API 문서에 맞게 조정한 코드 ###
### 2. GET_SEARCH ###

import boto3
import json # 1) json import
from collections import Counter

student_table = dynamodb.Table('Student')

def lambda_handler(event, context):
    # 쿼리스트링 파라미터 가져오기
    country = event.get('queryStringParameters', {}).get('country') # 2) event.get('queryStringParameters', {}).get('country') 형식 지킬 것
    major = event.get('queryStringParameters', {}).get('major')
    univ = event.get('queryStringParameters', {}).get('univ')

    # 대학 추천 함수 호출
    recommended_universities = recommend_universities(home_university=univ, major=major, preferred_country=country)

    # JSON 형식으로 상위 3개 학교 반환
    # return {
    #     'statusCode': 200,
    #     'schools': recommended_universities[:3]
    #     }
    # 3) return 값 형식 지킬것
    return {
    'statusCode': 200,
    'headers': {
        'Content-Type': 'application/json'
    },
    'body': json.dumps({'schools': recommended_universities[:3]}, ensure_ascii=False)
}

def recommend_universities(home_university=None, major=None, preferred_country=None):
    criteria = [
        {"univ": home_university, "major": major, "country": preferred_country},
        {"univ": home_university, "major": major},
        {"univ": home_university, "country": preferred_country},
        {"major": major, "country": preferred_country},
        {"country": preferred_country},
        {"major": major},
        {"univ": home_university}
    ]

    all_results = []  # 최종 추천 유학 학교 리스트
    found_universities = set()  # 중복 방지를 위한 집합

    for condition in criteria:
        # 필터링 조건 생성 (None 값 제거)
        condition = {k: v for k, v in condition.items() if v is not None}
        if not condition:
            continue

        # 필터 조건 생성
        filter_expression = " AND ".join([f"{key} = :{key}" for key in condition.keys()])
        expression_values = {f":{key}": value for key, value in condition.items()}

        response = student_table.scan(
            FilterExpression=filter_expression,
            ExpressionAttributeValues=expression_values
        )

        # 검색 결과가 있는 경우 foreign_universities 리스트에 추가
        if response['Items']:
            foreign_universities = [item['foreign_univ'] for item in response['Items']]
            university_counts = Counter(foreign_universities)

            # 학생 수 기준으로 내림차순 정렬
            for univ, count in university_counts.most_common():
                if univ not in found_universities:
                    all_results.append(univ)
                    found_universities.add(univ)

        # 최종적으로 3개 학교를 얻었다면 종료
        if len(all_results) >= 3:
            break

    return all_results[:3]


## 2. GET_INFO

In [ ]:
### API 문서에 맞게 조정한 코드 ###
### 3. GET_INFO ###

import boto3
from collections import Counter
import json

# DynamoDB 테이블 설정
student_table = dynamodb.Table('Student')
foreign_university_table = dynamodb.Table('ForeignUniversity')

# Lambda 핸들러 함수
def lambda_handler(event, context):
    # 쿼리스트링 파라미터 가져오기
    country = event.get('queryStringParameters', {}).get('country', '0')
    region = event.get('queryStringParameters', {}).get('region', '0')
    major = event.get('queryStringParameters', {}).get('major', '0')
    univ = event.get('queryStringParameters', {}).get('univ', '0')

    # '0'이나 None 값을 None으로 처리
    country = None if country == '0' or not country else country
    region = None if region == '0' or not region else region
    major = None if major == '0' or not major else major
    univ = None if univ == '0' or not univ else univ

    # 대학 필터링 함수 호출
    filtered_universities = filtered_universities_func(
        home_university=univ,
        major=major,
        preferred_country=country,
        preferred_region=region
    )

    # JSON 형식으로 상위 3개 학교 반환
    return {
    'statusCode': 200,
    'headers': {
        'Content-Type': 'application/json'
    },
    'body': json.dumps({"schools": filtered_universities["schools"]}, ensure_ascii=False)
    }

# 대학 필터링 함수
def filtered_universities_func(home_university=None, major=None, preferred_country=None, preferred_region=None):
    # 조건 설정 순서대로 우선순위 배열
    criteria = [
        {"univ": home_university, "major": major, "country": preferred_country, "region": preferred_region},
        {"univ": home_university, "major": major, "country": preferred_country},
        {"univ": home_university, "major": major, "region": preferred_region},
        {"univ": home_university, "country": preferred_country, "region": preferred_region},
        {"major": major, "country": preferred_country, "region": preferred_region},
        {"country": preferred_country, "region": preferred_region},
        {"univ": home_university, "major": major},
        {"univ": home_university, "country": preferred_country},
        {"univ": home_university, "region": preferred_region},
        {"major": major, "country": preferred_country},
        {"major": major, "region": preferred_region},
        {"country": preferred_country, "region": preferred_region},
        {"country": preferred_country},
        {"major": major},
        {"region": preferred_region},
        {"univ": home_university}
    ]

    all_results = []  # 모든 우선순위에서 얻은 결과를 저장할 리스트
    found_universities = set()  # 중복 방지를 위한 집합

    # Student 테이블에서 조건에 맞는 대학 리스트 추출
    for i, condition in enumerate(criteria):
        # 필터링 조건 설정 (None 값 제거)
        condition = {k: v for k, v in condition.items() if v is not None}
        if not condition:
            continue

        # 필터 조건 생성
        filter_expression = " AND ".join([f"#{key} = :{key}" if key == 'region' else f"{key} = :{key}" for key in condition.keys()])
        expression_values = {f":{key}": value for key, value in condition.items()}

        # 'region' 필드가 포함된 경우에만 ExpressionAttributeNames 설정
        if "region" in condition:
            expression_attribute_names = {"#region": "region"}
            response = student_table.scan(
                FilterExpression=filter_expression,
                ExpressionAttributeValues=expression_values,
                ExpressionAttributeNames=expression_attribute_names
            )
        else:
            response = student_table.scan(
                FilterExpression=filter_expression,
                ExpressionAttributeValues=expression_values
            )

        # 검색 결과가 있는 경우
        if response['Items']:
            foreign_universities = [item['foreign_univ'] for item in response['Items']]
            university_counts = Counter(foreign_universities)

            # 우선순위별로 학생이 많은 순서대로 정렬하여 추가
            for univ, count in university_counts.most_common():
                if univ not in found_universities:
                    # flag 설정: 완전 일치하는 첫 번째 조건일 때만 flag를 0으로 설정, 그 외는 1
                    flag_value = 0 if i == 0 else 1
                    all_results.append({
                        "id": univ,
                        "students": count,
                        # "flag": flag_value
                    })
                    found_universities.add(univ)

                # 상위 3개 학교를 찾으면 종료
                if len(all_results) >= 3:
                    break

        # 상위 3개 학교가 채워지면 전체 반복 종료
        if len(all_results) >= 3:
            break

    return {
        "schools": all_results[:3]
    }


## 3. GET_UNIV_LIST

In [ ]:
## 페이지별 12개씩 반환

import boto3
import json
from decimal import Decimal


# DynamoDB 테이블 설정
dormitory_table = dynamodb.Table('ForeignUniversity')

# Decimal을 JSON 직렬화 가능하게 변환하는 함수
def decimal_default(obj):
    if isinstance(obj, Decimal):
        return float(obj)  # 필요에 따라 int(obj)로 변환 가능
    raise TypeError

# Lambda 핸들러 함수
def lambda_handler(event, context):
    # 쿼리스트링 파라미터에서 나라, 지역 정보 가져오기
    query_params = event.get('queryStringParameters', {})
    country = query_params.get('country', '')  # 기본값 ''
    region = query_params.get('region', '').upper()    # 기본값 ''

    # `/univ/list?all` 요청 처리: country와 region이 모두 ''인 경우 전체 리스트 반환
    if country == '' and region == '':
        ForeignUniversity, total_count = get_filtered_ForeignUniversity()
    else:
        # 나라와 지역 필터를 통해 기숙사 정보 검색
        ForeignUniversity, total_count = get_filtered_ForeignUniversity(country, region)

    # JSON 형식으로 반환
    return {
        'statusCode': 200,
        'headers': {
            'Content-Type': 'application/json'
        },
        'body': json.dumps({
            "schools": ForeignUniversity,
            "totalData": total_count
        }, ensure_ascii=False, default=decimal_default)  # Decimal 값을 처리하기 위해 default 사용
    }

# 필터링된 기숙사 정보 조회 함수
def get_filtered_ForeignUniversity(country=None, region=None):
    # 필터 조건 설정
    filter_expression_parts = []
    expression_values = {}
    expression_attribute_names = {}

    if country and country != 'all' and country != '':
        filter_expression_parts.append("#country = :country")
        expression_values[":country"] = country
        expression_attribute_names["#country"] = "country"

    if region and region != 'all' and region != '':
        filter_expression_parts.append("#region = :region")
        expression_values[":region"] = region
        expression_attribute_names["#region"] = "region"

    # 전체 조건 설정 (필터링이 없는 경우 전체 데이터 반환)
    filter_expression = " AND ".join(filter_expression_parts) if filter_expression_parts else None

    # DynamoDB scan 요청
    scan_kwargs = {}
    if filter_expression:
        scan_kwargs['FilterExpression'] = filter_expression
        scan_kwargs['ExpressionAttributeValues'] = expression_values
        scan_kwargs['ExpressionAttributeNames'] = expression_attribute_names

    response = dormitory_table.scan(**scan_kwargs)

    # 기숙사 리스트 생성
    ForeignUniversity = []
    total_count = len(response['Items'])  # 전체 데이터 개수

    for item in response['Items']:
        ForeignUniversity.append({
            "id": item.get("name", "").upper(),
            "name": item.get("name", ""),
            "country": item.get("country", ""),
            "students": item.get("std_number", ""),
            "region": item.get("region", "").upper(),
            "img": item.get("image", "")
        })

    return ForeignUniversity, total_count


## 4. GET_UNIV

**반환시 return 값 정형화: 재직렬화한 코드**

In [ ]:
import boto3
import json
from decimal import Decimal

# DynamoDB 리소스 설정
university_table = dynamodb.Table('ForeignUniversity')

# JSON 직렬화 클래스
class DecimalEncoder(json.JSONEncoder):
    def default(self, o):
        if isinstance(o, Decimal):
            return float(o)
        return super(DecimalEncoder, self).default(o)

# JSON 문자열을 파싱하여 객체로 변환하는 함수
def parse_json_fields(item):
    # 필요한 필드를 파싱하고 JSON 객체로 변환
    for field in ['all_review', 'univ_info', 'review', 'major_info']:
        if field in item and isinstance(item[field], str):
            try:
                item[field] = json.loads(item[field])
            except json.JSONDecodeError:
                pass  # JSON 문자열이 아닌 경우 변환하지 않음
    return item

# Lambda 핸들러 함수
def lambda_handler(event, context):
    # 쿼리스트링 파라미터에서 학교 이름 가져오기
    university_name = event.get('queryStringParameters', {}).get('school')
    # 학교 정보 조회 함수 호출
    university_info = get_university_info(university_name)

    # JSON 형식으로 반환
    return {
        'statusCode': 200,
        'headers': {
            'Content-Type': 'application/json'
        },
        'body': json.dumps({"info": university_info}, ensure_ascii=False, cls=DecimalEncoder)
    }

# 학교 정보 조회 함수
def get_university_info(university_name):
    # 학교 이름이 정확히 일치하는 데이터 조회
    response = university_table.scan(
        FilterExpression="#name = :name",
        ExpressionAttributeNames={"#name": "name"},
        ExpressionAttributeValues={":name": university_name}
    )

    # 데이터가 존재하는 경우 지정된 JSON 구조로 변환하여 반환
    if 'Items' in response and response['Items']:
        item = parse_json_fields(response['Items'][0])  # 첫 번째 검색 결과만 사용한다고 가정

        # 요청된 반환 형식에 맞게 데이터 구조 변환
        return {
            "meta_data": {
                "location": item.get("region", "N/A"),
                "students": int(item.get("std_number", 0)),
                "homepage": item.get("url", "N/A")
            },
            "img_info": item.get("image", "N/A"),
            "ai_review": {
                "before_departure": item.get("review", {}).get("before_departure", "N/A"),
                "class": item.get("review", {}).get("class", "N/A"),
                "life": item.get("review", {}).get("life", "N/A"),
                "review": item.get("review", {}).get("review", "N/A")
            },
            "all_review": item.get("all_review", [])
        }
    else:
        # 데이터가 없는 경우 빈 객체 반환
        return {}


# 람다함수 테스트 코드 (API 연동시)
event = {
    "queryStringParameters": {
        "school": ""
    }
}
lambda_handler(event, None)


{'statusCode': 200,
 'headers': {'Content-Type': 'application/json'},
 'body': '{"info": {"meta_data": {"location": "보스턴", "students": 22000, "homepage": "https://harvard.edu"}, "img_info": "https://funiv.s3.amazonaws.com/ForeignUniversity/하버드대학교.jpg", "ai_review": {"before_departure": "N/A", "class": "N/A", "life": "N/A", "review": "N/A"}, "all_review": [{"before_departure": "준비사항", "class": "도착 및 수업", "life": "학교생활", "review": "총평"}]}}'}

## 5. GET_UNIV_URL

In [ ]:
# 인증 설정

import boto3
import json


# DynamoDB 테이블 설정
university_table = dynamodb.Table('ForeignUniversity')

# Lambda 핸들러 함수
def lambda_handler(event, context):
    # 쿼리스트링 파라미터에서 검색어 가져오기
    query = event.get('queryStringParameters', {}).get('query')

    # 학교 이름으로 검색 함수 호출
    search_result = search_universities_by_name(query)

    # JSON 형식으로 반환
    return {
    'statusCode': 200,
    'headers': {
        'Content-Type': 'application/json'
    },
    'body': json.dumps({"result": search_result}, ensure_ascii=False)
    }


# 검색어를 포함하는 학교 이름 검색 함수
def search_universities_by_name(name_substring):
    # 1. 이름에 해당 문자열이 포함된 학교 검색 (최대 3개)
    response = university_table.scan(
        FilterExpression="contains(#name, :name_substring)",
        ExpressionAttributeNames={"#name": "name"},
        ExpressionAttributeValues={":name_substring": name_substring}
    )

    # 2. 결과 리스트를 구성하여 최대 3개 반환
    result = []
    if response['Items']:
        for item in response['Items'][:3]:  # 최대 3개의 결과만 포함
            result.append({
                "id": item.get("name"),
                "img_url": item.get("image")
            })

    return result


## 6. GET_DORMITORY_LIST

In [ ]:
import boto3
import json
from decimal import Decimal


# DynamoDB 테이블 설정
dormitory_table = dynamodb.Table('Dormitory')

# Decimal을 JSON 직렬화 가능하게 변환하는 함수
def decimal_default(obj):
    if isinstance(obj, Decimal):
        return float(obj)  # 필요에 따라 int(obj)로 변환 가능
    raise TypeError

# Lambda 핸들러 함수
def lambda_handler(event, context):
    # 쿼리스트링 파라미터에서 나라와 지역 정보 가져오기
    query_params = event.get('queryStringParameters', {})
    country = query_params.get('country', '')  # 기본값 ''
    region = query_params.get('region', '')    # 기본값 ''

    # `/dorm/list?all` 요청 처리: country와 region이 모두 ''인 경우 전체 리스트 반환
    if country == '' and region == '':
        dormitories, total_count = get_filtered_dormitories()
    else:
        # 나라와 지역 필터를 통해 기숙사 정보 검색
        dormitories, total_count = get_filtered_dormitories(country, region)

    # JSON 형식으로 반환
    return {
        'statusCode': 200,
        'headers': {
            'Content-Type': 'application/json'
        },
        'body': json.dumps({
            "dormitory": dormitories,
            "totalData": total_count
        }, ensure_ascii=False, default=decimal_default)  # Decimal 값을 처리하기 위해 default 사용
    }

# 필터링된 기숙사 정보 조회 함수
def get_filtered_dormitories(country=None, region=None):
    # 필터 조건 설정
    filter_expression_parts = []
    expression_values = {}
    expression_attribute_names = {}

    if country and country != 'all' and country != '':
        filter_expression_parts.append("#country = :country")
        expression_values[":country"] = country
        expression_attribute_names["#country"] = "country"

    if region and region != 'all' and region != '':
        filter_expression_parts.append("#region = :region")
        expression_values[":region"] = region
        expression_attribute_names["#region"] = "region"

    # 전체 조건 설정 (필터링이 없는 경우 전체 데이터 반환)
    filter_expression = " AND ".join(filter_expression_parts) if filter_expression_parts else None

    # DynamoDB scan 요청
    scan_kwargs = {}
    if filter_expression:
        scan_kwargs['FilterExpression'] = filter_expression
        scan_kwargs['ExpressionAttributeValues'] = expression_values
        scan_kwargs['ExpressionAttributeNames'] = expression_attribute_names

    response = dormitory_table.scan(**scan_kwargs)

    # 기숙사 리스트 생성
    dormitories = []
    total_count = len(response['Items'])  # 전체 데이터 개수

    for item in response['Items']:
        dormitories.append({
            "id": item.get("name", "").upper(),
            "name": item.get("name", ""),
            "country": item.get("country", ""),
            "region": item.get("region", ""),
            "img": item.get("image", "")
        })

    return dormitories, total_count


## 7. GET_DORMITORY

In [ ]:
### API 문서에 맞게 조정한 코드 ###
### 8. GET_DORMITORY ###

import boto3
import json


# DynamoDB 테이블 설정
dormitory_table = dynamodb.Table('Dormitory')

# Lambda 핸들러 함수
def lambda_handler(event, context):
    # 쿼리스트링 파라미터에서 기숙사 이름 가져오기
    dormitory_name = event.get('queryStringParameters', {}).get('dom')

    # 기숙사 정보 조회 함수 호출
    dormitory_info = get_dormitory_info(dormitory_name)

    # JSON 형식으로 반환
    return {
    'statusCode': 200,
    'headers': {
        'Content-Type': 'application/json'
    },
    'body': json.dumps({"dormitory": dormitory_info}, ensure_ascii=False)
    }

# 기숙사 이름으로 기숙사 정보 조회 함수
def get_dormitory_info(dormitory_name):
    # 기숙사 이름이 정확히 일치하는 데이터 조회
    response = dormitory_table.scan(
        FilterExpression="#name = :name",
        ExpressionAttributeNames={"#name": "name"},
        ExpressionAttributeValues={":name": dormitory_name}
    )

    # 데이터가 존재하는 경우, 모든 항목을 리스트로 반환
    if 'Items' in response and response['Items']:
        return response['Items']  # 모든 항목 반환
    else:
        # 데이터가 없는 경우 빈 리스트 반환
        return []



## 8. GET_DORMITORY_URL

In [ ]:
# 인증 설정

import boto3
import json
from decimal import Decimal

# DynamoDB 리소스 설정
dormitory_table = dynamodb.Table('Dormitory')

# Decimal을 JSON 직렬화 가능한 타입으로 변환하는 클래스
class DecimalEncoder(json.JSONEncoder):
    def default(self, o):
        if isinstance(o, Decimal):
            return float(o)
        return super(DecimalEncoder, self).default(o)

# Lambda 핸들러 함수
def lambda_handler(event, context):
    # 쿼리스트링 파라미터에서 기숙사 이름 가져오기
    dormitory_name = event.get('queryStringParameters', {}).get('query')

    # 기숙사 정보 조회 함수 호출
    dormitory_info = get_dormitory_info(dormitory_name)

    # JSON 형식으로 반환
    return {
        'statusCode': 200,
        'headers': {
            'Content-Type': 'application/json'
        },
        'body': json.dumps({"info": dormitory_info}, ensure_ascii=False, cls=DecimalEncoder)
    }

def get_dormitory_info(dormitory_name):
    # 기숙사 이름이 포함된 데이터 조회 (대소문자 구분 없이)
    response = dormitory_table.scan(
        FilterExpression="contains(#name, :name)",
        ExpressionAttributeNames={"#name": "name"},
        ExpressionAttributeValues={":name": dormitory_name}
    )

    # 데이터가 존재하는 경우 지정된 JSON 구조로 변환하여 반환
    if 'Items' in response and response['Items']:
        item = response['Items'][0]  # 첫 번째 검색 결과만 사용한다고 가정

        # info 필드를 JSON 형식으로 바로 사용
        info_data = item.get("info", {})
        if isinstance(info_data, str):
            info_data = json.loads(info_data)

        return {
            "meta_data": {
                "location": item.get("region", "N/A"),
                "dorm_students": int(item.get("std_number", 0)),
                "homepage": item.get("url", "N/A")
            },
            "img_info": {
                "logo_img_Url": item.get("image", "N/A"),
                "map_img_url": item.get("location_image", "N/A")
            },
            "review": {
                "application": info_data.get("신청", "N/A"),
                "room": info_data.get("방 형태/스펙", "N/A"),
                "charge": info_data.get("금액", "N/A"),
                "location": info_data.get("위치", "N/A"),
                "review": info_data.get("후기", "N/A")
            }
        }
    else:
        return {}

ModuleNotFoundError: No module named 'boto3'